In [1]:
import numpy  as np
import pandas as pd
from skcredit.feature_discrete import SplitMixND, C1Discrete, CXDiscrete
from skcredit.feature_selector import SelectBins, SelectCIFE
from skcredit.linear_model     import LMClassifier, LMCreditcard
from sklearn.model_selection   import train_test_split
np.random.seed(7)

## 读入数据

[数据介绍](https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients)

In [2]:
dataset = pd.read_csv("../UCI_Credit_Card.csv")

In [3]:
dataset.shape

(30000, 25)

In [4]:
dataset.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,1,20000.0,2,2,1,24,2,2,-1,-1,-2,-2,3913.0,3102.0,689.0,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,0,2,2682.0,1725.0,2682.0,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,0,0,29239.0,14027.0,13559.0,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,0,0,46990.0,48233.0,49291.0,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,0,0,8617.0,5670.0,35835.0,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


In [5]:
cat_columns = ["SEX", "EDUCATION", "MARRIAGE"]
num_columns = ["LIMIT_BAL", "AGE",
               "PAY_0",     "PAY_2",     "PAY_3",     "PAY_4",     "PAY_5",     "PAY_6",
               "PAY_AMT1",  "PAY_AMT2",  "PAY_AMT3",  "PAY_AMT4",  "PAY_AMT5",  "PAY_AMT6",
               "BILL_AMT1", "BILL_AMT2", "BILL_AMT3", "BILL_AMT4", "BILL_AMT5", "BILL_AMT6"]

target = "default.payment.next.month"

In [6]:
# 分类变量需要 astype category
dataset[cat_columns] = dataset[cat_columns].astype("category")

## 分类特征

### 单特征分箱

In [7]:
# min_bin_cnt_negative=75
# min_bin_cnt_positive=75
# min_information_value_split_gain=0.005

smnd = SplitMixND()
smnd.fit(dataset[["EDUCATION"]], dataset["default.payment.next.month"])

SplitMixND()

In [8]:
smnd.build_table()

,Column,Bucket,CntPositive,CntNegative,PctPositive,PctNegative,WoE,IvS
0,EDUCATION,"[0, 1, 4, 5, 6]",2069,8984,0.311784,0.384523,-0.209693,0.015253
1,EDUCATION,"[2, 3]",4567,14380,0.688216,0.615477,0.111705,0.008125
2,EDUCATION,[MISSING],0,0,0.000000,0.000000,0.0,0.000000
0,ALL,ALL,6636,23364,1.000000,1.000000,-,0.023378


### 多特征分箱

In [9]:
smnd = SplitMixND()
smnd.fit(dataset[["EDUCATION", "SEX"]], dataset["default.payment.next.month"])

SplitMixND()

In [10]:
smnd.build_table()

,Column,Bucket,CntPositive,CntNegative,PctPositive,PctNegative,WoE,IvS
0,EDUCATION @ SEX,"[0, 1, 4, 5, 6] @ [1, 2]",2069,8984,0.311784,0.384523,-0.209693,0.015253
1,EDUCATION @ SEX,"[2, 3] @ [2]",2614,8969,0.393912,0.383881,0.025794,0.000259
2,EDUCATION @ SEX,"[2, 3] @ [1]",1953,5411,0.294304,0.231596,0.23962,0.015026
3,EDUCATION @ SEX,"[0, 1, 2, 3, 4, 5, 6] @ [MISSING]",0,0,0.000000,0.000000,0.0,0.000000
4,EDUCATION @ SEX,"[MISSING] @ [1, 2]",0,0,0.000000,0.000000,0.0,0.000000
5,EDUCATION @ SEX,[MISSING] @ [MISSING],0,0,0.000000,0.000000,0.0,0.000000
0,ALL,ALL,6636,23364,1.000000,1.000000,-,0.030538


In [11]:
smnd = SplitMixND()
smnd.fit(dataset[["EDUCATION", "SEX", "MARRIAGE"]], dataset["default.payment.next.month"])

SplitMixND()

In [12]:
smnd.build_table()

,Column,Bucket,CntPositive,CntNegative,PctPositive,PctNegative,WoE,IvS
0,EDUCATION @ SEX @ MARRIAGE,"[0, 1, 4, 5, 6] @ [1, 2] @ [0, 1, 2, 3]",2069,8984,0.311784,0.384523,-0.209693,0.015253
1,EDUCATION @ SEX @ MARRIAGE,"[2, 3] @ [2] @ [0, 1, 2, 3]",2614,8969,0.393912,0.383881,0.025794,0.000259
2,EDUCATION @ SEX @ MARRIAGE,"[2, 3] @ [1] @ [0, 1, 2, 3]",1953,5411,0.294304,0.231596,0.23962,0.015026
3,EDUCATION @ SEX @ MARRIAGE,"[0, 1, 2, 3, 4, 5, 6] @ [1, 2] @ [MISSING]",0,0,0.000000,0.000000,0.0,0.000000
4,EDUCATION @ SEX @ MARRIAGE,"[0, 1, 2, 3, 4, 5, 6] @ [MISSING] @ [0, 1, 2, 3]",0,0,0.000000,0.000000,0.0,0.000000
5,EDUCATION @ SEX @ MARRIAGE,"[0, 1, 2, 3, 4, 5, 6] @ [MISSING] @ [MISSING]",0,0,0.000000,0.000000,0.0,0.000000
6,EDUCATION @ SEX @ MARRIAGE,"[MISSING] @ [1, 2] @ [0, 1, 2, 3]",0,0,0.000000,0.000000,0.0,0.000000
7,EDUCATION @ SEX @ MARRIAGE,"[MISSING] @ [1, 2] @ [MISSING]",0,0,0.000000,0.000000,0.0,0.000000
8,EDUCATION @ SEX @ MARRIAGE,"[MISSING] @ [MISSING] @ [0, 1, 2, 3]",0,0,0.000000,0.000000,0.0,0.000000
9,EDUCATION @ SEX @ MARRIAGE,[MISSING] @ [MISSING] @ [MISSING],0,0,0.000000,0.000000,0.0,0.000000


## 连续特征

### 单特征分箱

In [13]:
# min_bin_cnt_negative=75
# min_bin_cnt_positive=75
# min_information_value_split_gain=0.005

smnd = SplitMixND()
smnd.fit(dataset[["PAY_AMT1"]], dataset["default.payment.next.month"])

SplitMixND()

In [14]:
smnd.build_table()

,Column,Bucket,CntPositive,CntNegative,PctPositive,PctNegative,WoE,IvS
0,PAY_AMT1,"(-inf, 13.000000]",1930,3453,0.290838,0.147791,0.676964,0.096837
1,PAY_AMT1,"(13.000000, 4501.000000]",3487,12426,0.525467,0.531844,-0.012062,0.000077
2,PAY_AMT1,"(4501.000000, 25794.000000]",1136,6585,0.171187,0.281844,-0.498594,0.055173
3,PAY_AMT1,"(25794.000000, +inf)",83,900,0.012508,0.038521,-1.124867,0.029261
4,PAY_AMT1,[MISSING],0,0,0.000000,0.000000,0.0,0.000000
0,ALL,ALL,6636,23364,1.000000,1.000000,-,0.181348


### 多特征分箱

In [15]:
smnd = SplitMixND()
smnd.fit(dataset[["PAY_AMT1", "PAY_AMT2"]], dataset["default.payment.next.month"])

SplitMixND()

In [16]:
smnd.build_table()

,Column,Bucket,CntPositive,CntNegative,PctPositive,PctNegative,WoE,IvS
0,PAY_AMT1 @ PAY_AMT2,"(-inf, 13.000000] @ (-inf, +inf)",1930,3453,0.290838,0.147791,0.676964,0.096837
1,PAY_AMT1 @ PAY_AMT2,"(13.000000, +inf) @ (-inf, 33.000000]",972,1945,0.146474,0.083248,0.565026,0.035724
2,PAY_AMT1 @ PAY_AMT2,"(13.000000, 4739.000000] @ (33.000000, 2000.00...",1606,5580,0.242013,0.238829,0.013245,0.000042
3,PAY_AMT1 @ PAY_AMT2,"(4739.000000, +inf) @ (33.000000, 2000.000000]",110,733,0.016576,0.031373,-0.637978,0.009440
4,PAY_AMT1 @ PAY_AMT2,"(13.000000, 3000.000000] @ (2000.000000, 15043...",790,3270,0.119048,0.139959,-0.161825,0.003384
5,PAY_AMT1 @ PAY_AMT2,"(3000.000000, +inf) @ (2000.000000, 15043.000000]",1103,6769,0.166215,0.289719,-0.555633,0.068623
6,PAY_AMT1 @ PAY_AMT2,"(13.000000, +inf) @ (15043.000000, +inf)",125,1614,0.018837,0.069081,-1.29947,0.065291
7,PAY_AMT1 @ PAY_AMT2,"(-inf, +inf) @ [MISSING]",0,0,0.000000,0.000000,0.0,0.000000
8,PAY_AMT1 @ PAY_AMT2,"[MISSING] @ (-inf, +inf)",0,0,0.000000,0.000000,0.0,0.000000
9,PAY_AMT1 @ PAY_AMT2,[MISSING] @ [MISSING],0,0,0.000000,0.000000,0.0,0.000000


In [17]:
smnd = SplitMixND()
smnd.fit(dataset[["PAY_AMT1", "PAY_AMT2", "PAY_AMT3"]], dataset["default.payment.next.month"])

SplitMixND()

In [18]:
smnd.build_table()

,Column,Bucket,CntPositive,CntNegative,PctPositive,PctNegative,WoE,IvS
0,PAY_AMT1 @ PAY_AMT2 @ PAY_AMT3,"(-inf, 13.000000] @ (-inf, +inf) @ (-inf, +inf)",1930,3453,0.290838,0.147791,0.676964,0.096837
1,PAY_AMT1 @ PAY_AMT2 @ PAY_AMT3,"(13.000000, +inf) @ (-inf, 33.000000] @ (-inf,...",972,1945,0.146474,0.083248,0.565026,0.035724
2,PAY_AMT1 @ PAY_AMT2 @ PAY_AMT3,"(13.000000, +inf) @ (33.000000, 2000.000000] @...",287,612,0.043249,0.026194,0.501437,0.008552
3,PAY_AMT1 @ PAY_AMT2 @ PAY_AMT3,"(13.000000, 4739.000000] @ (33.000000, 2000.00...",1343,5060,0.202381,0.216573,-0.067774,0.000962
4,PAY_AMT1 @ PAY_AMT2 @ PAY_AMT3,"(4739.000000, +inf) @ (33.000000, 2000.000000]...",86,641,0.012960,0.027435,-0.749995,0.010857
5,PAY_AMT1 @ PAY_AMT2 @ PAY_AMT3,"(13.000000, +inf) @ (2000.000000, +inf) @ (-in...",369,871,0.055606,0.037280,0.399842,0.007328
6,PAY_AMT1 @ PAY_AMT2 @ PAY_AMT3,"(13.000000, 3803.000000] @ (2000.000000, 15043...",758,3792,0.114225,0.162301,-0.351278,0.016888
7,PAY_AMT1 @ PAY_AMT2 @ PAY_AMT3,"(3803.000000, +inf) @ (2000.000000, 15043.0000...",789,5484,0.118897,0.234720,-0.680137,0.078776
8,PAY_AMT1 @ PAY_AMT2 @ PAY_AMT3,"(13.000000, +inf) @ (15043.000000, +inf) @ (8....",102,1506,0.015371,0.064458,-1.433553,0.070369
9,PAY_AMT1 @ PAY_AMT2 @ PAY_AMT3,"(-inf, +inf) @ (-inf, +inf) @ [MISSING]",0,0,0.000000,0.000000,0.0,0.000000


## 并行单特征分箱

In [19]:
trn_x, tst_x, trn_y, tst_y = train_test_split(
        dataset[cat_columns + num_columns], dataset[target], train_size=0.75, shuffle=True, random_state=7)

In [20]:
trn_x = trn_x.reset_index(drop=True)
tst_x = tst_x.reset_index(drop=True)
trn_y = trn_y.reset_index(drop=True)
tst_y = tst_y.reset_index(drop=True)

In [21]:
c1 = C1Discrete(keep_columns=[], date_columns=[])
c1.fit(trn_x, trn_y)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done   2 out of  23 | elapsed:    2.8s remaining:   30.3s
[Parallel(n_jobs=-1)]: Done   4 out of  23 | elapsed:    2.8s remaining:   13.9s
[Parallel(n_jobs=-1)]: Done   6 out of  23 | elapsed:    2.9s remaining:    8.3s
[Parallel(n_jobs=-1)]: Done   8 out of  23 | elapsed:    2.9s remaining:    5.5s
[Parallel(n_jobs=-1)]: Done  10 out of  23 | elapsed:    3.0s remaining:    3.9s
[Parallel(n_jobs=-1)]: Done  12 out of  23 | elapsed:    3.2s remaining:    3.0s
[Parallel(n_jobs=-1)]: Done  14 out of  23 | elapsed:    3.3s remaining:    2.1s
[Parallel(n_jobs=-1)]: Done  16 out of  23 | elapsed:    3.3s remaining:    1.4s
[Parallel(n_jobs=-1)]: Done  18 out of  23 | elapsed:    3.3s remaining:    0.9s
[Parallel(n_jobs=-1)]: Done  20 out of  23 | elapsed:    3.4s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done  23 out of  23 | elapse

C1Discrete(date_columns=[], keep_columns=[])

In [22]:
c1.information_value_score

,IvS
SEX,0.008907
EDUCATION,0.027667
MARRIAGE,0.000000
LIMIT_BAL,0.179315
AGE,0.014110
PAY_0,0.874626
PAY_2,0.563306
PAY_3,0.421203
PAY_4,0.371034
PAY_5,0.336415


In [23]:
c1.information_value_table

,Column,Bucket,CntPositive,CntNegative,PctPositive,PctNegative,WoE,IvS
0,SEX,[2],2824,10735,0.566499,0.612903,-0.078731,0.003653
1,SEX,[1],2161,6780,0.433501,0.387097,0.113218,0.005254
2,SEX,[MISSING],0,0,0.000000,0.000000,0.0,0.000000
0,ALL,ALL,4985,17515,1.000000,1.000000,-,0.008907
0,EDUCATION,"[0, 1, 4, 5]",1529,6756,0.306720,0.385727,-0.229193,0.018108
1,EDUCATION,"[2, 3, 6]",3456,10759,0.693280,0.614273,0.120993,0.009559
2,EDUCATION,[MISSING],0,0,0.000000,0.000000,0.0,0.000000
0,ALL,ALL,4985,17515,1.000000,1.000000,-,0.027667
0,MARRIAGE,"[0, 1, 2, 3]",4985,17515,1.000000,1.000000,0.0,0.000000
1,MARRIAGE,[MISSING],0,0,0.000000,0.000000,0.0,0.000000


In [24]:
trn_x_c1 = c1.transform(trn_x)
tst_x_c1 = c1.transform(tst_x)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0250s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   2 out of  23 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done   4 out of  23 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of  23 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   8 out of  23 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  23 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  12 out of  23 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  14 out of  23 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  16 out of  23 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  18 out of  23 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  20 out of  23 | elap

## 并行多特征分箱

In [25]:
smnd1 = SplitMixND()
smnd1.fit(trn_x[["PAY_AMT1", "PAY_AMT2"]], trn_y)

SplitMixND()

In [26]:
smnd2 = SplitMixND()
smnd2.fit(trn_x[["PAY_AMT3", "PAY_AMT4"]], trn_y)

SplitMixND()

In [27]:
cx = CXDiscrete(
    keep_columns=[], 
    date_columns=[], 
    feature_spliter={"PAY_AMT1 @ PAY_AMT2": smnd1,
                     "PAY_AMT3 @ PAY_AMT4": smnd2}
)

In [28]:
cx.fit(trn_x[["PAY_AMT1", "PAY_AMT2", "PAY_AMT3", "PAY_AMT4"]], trn_y)

CXDiscrete(date_columns=[],
           feature_spliter={'PAY_AMT1 @ PAY_AMT2': SplitMixND(),
                            'PAY_AMT3 @ PAY_AMT4': SplitMixND()},
           keep_columns=[])

In [29]:
cx.information_value_score

,IvS
PAY_AMT1 @ PAY_AMT2,0.276458
PAY_AMT3 @ PAY_AMT4,0.182654


In [30]:
cx.information_value_table

,Column,Bucket,CntPositive,CntNegative,PctPositive,PctNegative,WoE,IvS
0,PAY_AMT1 @ PAY_AMT2,"(-inf, 8.000000] @ (-inf, +inf)",1459,2573,0.292678,0.146903,0.689303,0.100483
1,PAY_AMT1 @ PAY_AMT2,"(8.000000, +inf) @ (-inf, 24.000000]",700,1461,0.140421,0.083414,0.520828,0.029691
2,PAY_AMT1 @ PAY_AMT2,"(8.000000, 3800.000000] @ (24.000000, 4814.000...",1783,6509,0.357673,0.371624,-0.038264,0.000534
3,PAY_AMT1 @ PAY_AMT2,"(3800.000000, +inf) @ (24.000000, 4814.000000]",371,2080,0.074423,0.118755,-0.467297,0.020716
4,PAY_AMT1 @ PAY_AMT2,"(8.000000, +inf) @ (4814.000000, 15000.000000]",582,3656,0.116750,0.208735,-0.58103,0.053446
5,PAY_AMT1 @ PAY_AMT2,"(8.000000, +inf) @ (15000.000000, +inf)",90,1236,0.018054,0.070568,-1.363202,0.071587
6,PAY_AMT1 @ PAY_AMT2,"(-inf, +inf) @ [MISSING]",0,0,0.000000,0.000000,0.0,0.000000
7,PAY_AMT1 @ PAY_AMT2,"[MISSING] @ (-inf, +inf)",0,0,0.000000,0.000000,0.0,0.000000
8,PAY_AMT1 @ PAY_AMT2,[MISSING] @ [MISSING],0,0,0.000000,0.000000,0.0,0.000000
0,ALL,ALL,4985,17515,1.000000,1.000000,-,0.276458


In [31]:
trn_x_cx = cx.transform(trn_x)
tst_x_cx = cx.transform(tst_x)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0370s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0180s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    0.0s finished


In [32]:
trn_x = pd.concat([trn_x_c1, trn_x_cx], axis="columns")
tst_x = pd.concat([tst_x_c1, tst_x_cx], axis="columns")

## 特征选择

In [33]:
select = SelectBins(
    keep_columns=[], 
    date_columns=[],
)
select.fit(trn_x,        trn_y)
trn_x = select.transform(trn_x)
tst_x = select.transform(tst_x)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0350s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of  25 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done   6 out of  25 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  25 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  12 out of  25 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  14 out of  25 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  16 out of  25 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  18 out of  25 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  20 out of  25 | elapsed:    0.0s remain

In [34]:
select = SelectCIFE(
    keep_columns=[], 
    date_columns=[],
    nums_columns=10
)
select.fit(trn_x,        trn_y)
trn_x = select.transform(trn_x)
tst_x = select.transform(tst_x)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0130s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  14 ta

In [39]:
trn_x.head()

,SEX,LIMIT_BAL,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,PAY_AMT1,PAY_AMT2
0,0.113218,0.178152,-0.578847,-0.386973,-0.318642,-0.265879,-0.228131,-0.211054,-0.489777,-1.211327
1,0.113218,0.178152,-0.578847,-0.386973,-0.318642,-0.265879,-0.228131,1.345301,-0.007242,0.018808
2,0.113218,-0.360787,-0.578847,-0.386973,-0.318642,-0.265879,-0.228131,-0.211054,-0.489777,-0.416921
3,-0.078731,0.178152,2.094992,-0.386973,-0.318642,-0.265879,-0.228131,-0.211054,-0.007242,-0.416921
4,-0.078731,-0.759759,-0.578847,-0.386973,-0.318642,-0.265879,-0.228131,-0.211054,-0.489777,-1.211327


## 模型训练

In [44]:
lm = LMClassifier(keep_columns=[], date_columns=[])
lm.fit(trn_x, trn_y)
print("trn ks {}".format(lmclassifier.score(trn_x, trn_y)))
print("tst ks {}".format(lmclassifier.score(tst_x, tst_y)))

trn ks 0.41052
tst ks 0.39004


In [45]:
lm.model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Generalized Linear Model Regression Results                      
======================================================================================
Dep. Variable:     default.payment.next.month   No. Observations:                22500
Model:                                    GLM   Df Residuals:                    22495
Model Family:                        Binomial   Df Model:                            4
Link Function:                          Logit   Scale:                          1.0000
Method:                                  IRLS   Log-Likelihood:                -10007.
Date:                        Fri, 03 Jun 2022   Deviance:                       20014.
Time:                                18:05:50   Pearson chi2:                 2.23e+04
No. Iterations:                             4   Pseudo R-squ. (CS):             0.1548
Covariance Type:                    nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.2451      0.018    -69.342      0.000      -1.280      -1.210
PAY_2          0.1507      0.033      4.567      0.000       0.086       0.215
PAY_4          0.3009      0.038      7.967      0.000       0.227       0.375
PAY_0          0.7828      0.023     34.063      0.000       0.738       0.828
PAY_3          0.2403      0.039      6.112      0.000       0.163       0.317
==============================================================================
"""

## 评分卡生成

In [48]:
lmcreditcard = LMCreditcard(
        [], [], c1, cx, lm, BASE=500,  PDO=20,  ODDS=1)
lmcreditcard.show_scorecard()

,Column,Bucket,WoE,Coefficients,PartialScore,OffsetScores
0,PAY_2,"(-inf, 1.000000]",-0.386973,0.150750,1.683222,535.926479
1,PAY_2,"(1.000000, +inf)",1.525134,0.150750,-6.633904,535.926479
2,PAY_2,[MISSING],0.0,0.150750,-0.0,535.926479
0,PAY_4,"(-inf, 1.000000]",-0.265879,0.300906,2.308441,535.926479
1,PAY_4,"(1.000000, 2.000000]",1.358094,0.300906,-11.791374,535.926479
2,PAY_4,"(2.000000, +inf)",1.9612,0.300906,-17.027723,535.926479
3,PAY_4,[MISSING],0.0,0.300906,-0.0,535.926479
0,PAY_0,"(-inf, 0.000000]",-0.578847,0.782783,13.074038,535.926479
1,PAY_0,"(0.000000, 1.000000]",0.615374,0.782783,-13.89905,535.926479
2,PAY_0,"(1.000000, +inf)",2.094992,0.782783,-47.318187,535.926479
